## Régression polynomiale

### Premier exemple

In [ ]:
sns.scatterplot(data=df, x="Income", y="MntWines", color="black")

In [ ]:
sns.scatterplot(x=df["Income"] ** 2, y=df["MntWines"], color="black")

In [ ]:
sns.regplot(
    x=df["Income"],
    y=df["MntWines"],
    scatter_kws={"color": "black"},
    line_kws={"color": "red"},
)

In [ ]:
sns.regplot(
    x=df["Income"] ** 2,
    y=df["MntWines"],
    scatter_kws={"color": "black"},
    line_kws={"color": "red"},
)

In [ ]:
sns.regplot(
    x=df["Income"] ** 3,
    y=df["MntWines"],
    scatter_kws={"color": "black"},
    line_kws={"color": "red"},
)

### Afficher la courbe

In [ ]:
# Régression linéaire basique
from sklearn.linear_model import LinearRegression

In [ ]:
X = np.array(df[["Income"]] ** 2)
y = np.array(df[["MntWines"]])

In [ ]:
reg = LinearRegression().fit(X, y)

In [ ]:
# Tracer les données et la courbe de régression polynomiale
plt.scatter(df["Income"], y, label="Données", color="black")
plt.scatter(df["Income"], reg.predict(X), label="Régression polynomiale", color="red")

plt.legend(loc="best")
plt.title("Régression polynomiale d'un cube de données synthétiques")

plt.show()

### Polynomial Features

In [ ]:
# Pour la régression polynomiale
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# Transformer les données en matrice de caractéristiques polynomiales
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

Si on donne une variable $V$, PolynomialFeatures crée les variables
- $1$
- $V$
- $V^2$

Si on donne les variables $V$ et $W$, PolynomialFeatures crée les variables
- $1$
- $V$
- $W$
- $V \times W$
- $V^2$
- $W^2$

(attention à l'explosion combinatoire du nombre de variables !)

In [ ]:
# Entraîner un modèle de régression linéaire sur les données transformées
reg = LinearRegression().fit(X_poly, y)

In [ ]:
# Tracer les données et la courbe de régression polynomiale
plt.scatter(df["Income"], y, label="Données", color="black")
plt.scatter(
    df["Income"], reg.predict(X_poly), label="Régression polynomiale", color="red"
)

plt.legend(loc="best")
plt.title("Régression polynomiale d'un cube de données synthétiques")

plt.show()

### Attention au sur-apprentissage

In [ ]:
# Transformer les données en matrice de caractéristiques polynomiales
poly = PolynomialFeatures(degree=10)
X_poly = poly.fit_transform(X)

In [ ]:
# Entraîner un modèle de régression linéaire sur les données transformées
reg = LinearRegression().fit(X_poly, y)

In [ ]:
# Tracer les données et la courbe de régression polynomiale
plt.scatter(df["Income"], y, label="Données", color="black")
plt.scatter(
    df["Income"], reg.predict(X_poly), label="Régression polynomiale", color="red"
)

plt.legend(loc="best")
plt.title("Régression polynomiale d'un cube de données synthétiques")

plt.show()

# Transformation des données

In [ ]:
df_transforme = df.copy()

## Mise à l'échelle (réduction ?)

### Explication

In [ ]:
# todo: expliquer pourquoi on a envie de le faire

### Sélection des variables

In [ ]:
df[var_numeriques].hist(figsize=(12, 12), bins=30)
plt.show()

In [ ]:
# todo(prof): est-ce que notre choix de variables est bon ?
var_a_scaler = [
    "Year_Birth",
    "Income",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
]

In [ ]:
df["Income"].hist()

In [ ]:
(df["Income"] / df["Income"].std()).hist()

### Transformation

In [ ]:
for var in var_a_scaler:
    df_transforme[var] = df[var] / df[var].std()
    df_transforme[var] -= df_transforme[var].min()

### Conclusion

In [ ]:
df_transforme[var_numeriques].hist(figsize=(12, 12), bins=30)
plt.show()

Les changements sur la matrice de corrélation sont de l'ordre de $10^{-15}$, donc extrêmement négligeables.

In [ ]:
sns.heatmap(df_transforme[var_numeriques].corr() - df[var_numeriques].corr())

## Normalisation

### Explication

Pour normaliser les données, nous allons utiliser la transformation de Box-Cox, définie $\forall x > 0, $ comme ci-dessous :
$B(x, \lambda) = \begin{cases} \frac{x^{\lambda} - 1}{\lambda} & \text{  si } \lambda \neq 0 \\ \log(x) & \text{  si } \lambda = 0 \end{cases}$

Cette transformation est à appliquer à une variable (strictement positive), en ajustant le $\lambda$ pour maximiser la normalité.

Nous allons utiliser la librairie `scipy.stats.boxcox` qui estime le meilleur paramètre $\lambda$.

### Sélection des variables

In [ ]:
df[var_numeriques].hist(figsize=(12, 12), bins=30)
plt.show()

In [ ]:
# todo(prof): faut-il inclure la distribution uniforme Recency ?
# todo(prof): est-ce que notre choix de variables est bon ?
var_a_normaliser = [
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
]

### Transformation

In [ ]:
sns.histplot(df["MntWines"], kde=True)

In [ ]:
from scipy.stats import boxcox

In [ ]:
boxcox_lambdas = {}  # on garde les lambdas, pour la transformation inverse

In [ ]:
for var in var_a_normaliser:
    var_strict_positif = df[var] + df[var].min() + 1

    var_apres_boxcox, l = boxcox(var_strict_positif)

    df_transforme[var] = var_apres_boxcox
    boxcox_lambdas[var] = l

In [ ]:
# Tuto : comment récupérer la fonction initiale
# (il faut avoir récupéré le paramètre "l" lambda)

# from scipy.special import inv_boxcox
# initial = inv_boxcox(incbox, l)
# initial = pd.DataFrame(initial)
# sns.histplot(initial, bins=50, kde=True)

In [ ]:
sns.histplot(df_transforme["MntWines"], bins=50, kde=True)

In [ ]:
df_transforme[var_numeriques].hist(figsize=(12, 12), bins=30)
plt.show()

### Conclusion

In [ ]:
# todo: analyser les résultats, notamment l'impact sur la matrice de corrélation (du coup, mettre ci-dessous la matrice de corrélation pré-transformation et comparer)

In [ ]:
sns.heatmap(
    df_transforme[var_numeriques].corr() - df[var_numeriques].corr(),
    cmap="BrBG",
    vmin=-1,
    vmax=1,
)

In [ ]:
df_transforme[var_numeriques].corr() - df[var_numeriques].corr()

In [ ]:
plt.figure(figsize=(12, 12))
plt.title("Matrice de corrélation des variables numériques avant transformation")
sns.heatmap(
    df[var_numeriques].corr()[df[var_numeriques].corr().abs() > 0.5],
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

In [ ]:
plt.figure(figsize=(12, 12))
plt.title("Matrice de corrélation des variables numériques après transformation")
sns.heatmap(
    df_transforme[var_numeriques].corr()[
        df_transforme[var_numeriques].corr().abs() > 0.5
    ],
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

## Variables catégoriques

### Encodage

In [ ]:
# todo: One Hot Encoding, Target Encoding, etc

# Analyse multi-variée

## Clustering

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_apres_scale = pd.DataFrame(
    scaler.fit_transform(df_transforme[var_numeriques]),
    columns=df[var_numeriques].columns,
)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
k3 = KMeans(n_clusters=3)
k4 = KMeans(n_clusters=4)

In [ ]:
k3.fit(df_apres_scale[var_numeriques])
k4.fit(df_apres_scale[var_numeriques])

In [ ]:
df["cluster"] = k3.labels_

In [ ]:
sns.histplot(df["cluster"])

In [ ]:
sns.histplot(df, x="Income", hue="cluster", kde=True)

In [ ]:
sns.scatterplot(df, x="Year_Birth", y="Income", hue="cluster")

In [ ]:
df[var_numeriques + ["cluster"]]

In [ ]:
df["cluster"] = k4.labels_
sns.histplot(df["cluster"])

In [ ]:
sns.histplot(df, x="Income", hue="cluster", kde=True)

In [ ]:
sns.scatterplot(df, x="Year_Birth", y="Income", hue="cluster")

## Analyse en Composantes Principales (ACP)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# todo: centrer / réduire

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
df_apres_scale = pd.DataFrame(
    scaler.fit_transform(df[var_numeriques]), columns=df[var_numeriques].columns
)

In [ ]:
df_apres_scale

In [ ]:
# for var in var_numeriques:
#     _, ax = plt.subplots(1, 2, figsize=(8, 2))
#     sns.boxplot(df_apres_scale[var], width=0.25, ax=ax[0])
#     sns.histplot(df_apres_scale[var], kde=True, ax=ax[1])
#     plt.show()

In [ ]:
acp = PCA(random_state=0)

In [ ]:
acp.fit(df_apres_scale)

In [ ]:
variance_expliquee = pd.Series(
    acp.explained_variance_ratio_, index=df[var_numeriques].columns
)

In [ ]:
variance_expliquee

In [ ]:
variance_expliquee.plot.barh()

In [ ]:
df_acp = pd.DataFrame(acp.fit_transform(df_apres_scale), index=df.index)

In [ ]:
df_acp.head()

In [ ]:
sns.scatterplot(df_acp, x=0, y=1)

### Cercle de corrélations

In [ ]:
from mlxtend.plotting import plot_pca_correlation_graph

In [ ]:
_, correlation_matrix = plot_pca_correlation_graph(
    df_apres_scale,
    df_apres_scale.columns,
    X_pca=df_acp.iloc[:, :2],
    explained_variance=acp.explained_variance_[:2],
    dimensions=(1, 2),
)

In [ ]:
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

### Test clusters

In [ ]:
df_acp["clusterk3"] = k3.labels_
df_acp["clusterk4"] = k4.labels_

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue="clusterk3")

In [ ]:
df_acp.head()

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue=df["Response"])

In [ ]:
sns.scatterplot(df_acp, x=0, y=2, hue=df["Response"])

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue="clusterk4")

## Analyse Factorielle des Correspondances (AFC)

In [ ]:
import prince

In [ ]:
table_contingence = pd.crosstab(df["Kidhome"], df["Teenhome"])

In [ ]:
table_contingence

In [ ]:
ca = prince.CA(
    # n_components=3,
    # n_iter=3,
    # copy=True,
    # check_input=True,
    # engine='sklearn',
    random_state=0
)

ca = ca.fit(table_contingence)

In [ ]:
ca.eigenvalues_summary

In [ ]:
ca.plot(table_contingence)

In [ ]:
table_contingence = pd.crosstab(df["Marital_Status"], df["Education"])

In [ ]:
table_contingence

In [ ]:
ca = prince.CA(random_state=0)
ca = ca.fit(table_contingence)

In [ ]:
ca.eigenvalues_summary

In [ ]:
ca.plot(table_contingence)

In [ ]:
# todo: à interpréter

## Analyse des Correspondances Multiples (ACM)

In [ ]:
mca = prince.MCA(random_state=0)
mca = mca.fit(df[var_categoriques])

In [ ]:
mca.plot(df[var_categoriques])

In [ ]:
# todo: à interpréter

# Régressions

In [ ]:
from sklearn.model_selection import train_test_split

## Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)

In [ ]:
X = df[var_numeriques]

In [ ]:
y = df[["Response"]].astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
log_reg = LogisticRegression(random_state=0)

In [ ]:
log_reg.fit(X_train, y_train)

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=log_reg.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=log_reg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=log_reg.classes_)
disp.plot()

In [ ]:
# todo: commenter

## Régression PLS

In [ ]:
# todo: refaire la régression PLS mais PAS sur une variable catégorique (erreur ici)

In [ ]:
from sklearn.cross_decomposition import PLSRegression

In [ ]:
# Initialisation de l'objet PLSRegression avec 2 composantes PLS
pls = PLSRegression(n_components=2)

In [ ]:
# Apprentissage du modèle sur les données
pls.fit(df_transforme[var_numeriques], df_transforme["Response"])

In [ ]:
# Prédiction de la variable cible sur de nouvelles données
y_pred = pls.predict(df_transforme[var_numeriques])

In [ ]:
# Evaluation de la performance du modèle
r2 = pls.score(df_transforme[var_numeriques], df_transforme["Response"].astype(int))

In [ ]:
r2

## Régression linéaire simple

Modèle simple : une variable à expliquer Y et une seule variable explicative X.  

$$y_i = \beta_0 + \beta_1 X_i + \epsilon_i$$

Hypothèses à vérifier pour la régression linéaire simple :  

1) il existe une corrélation linéaire entre X et Y

1) la distribution de l’erreur ε est indépendante de la variable X (exogénéité)

2) l'erreur suit une loi normale centrée i.e. E(ε) = 0

3) l’erreur est de variance constante (homoscédasticité)
i.e Var(εi) = s, s une constante

4) les erreurs sont indépendantes (absence d'autocorrélation)
i.e. Cov(εi, εj) = 0, pour tout i, j

In [ ]:
X = np.array(df_transforme["Income"])
Y = np.array(df_transforme["NumStorePurchases"])

### Hypothèse 1 : corrélation linéaire

In [ ]:
from scipy.stats import pearsonr

In [ ]:
print(
    "Le coefficient de corrélation linéaire entre X et Y vaut",
    pearsonr(X, Y)[0],
    "la pvalue associée vaut",
    pearsonr(X, Y)[1],
    "il existe donc bien une relation linéaire entre X et Y.",
)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
model = LinearRegression().fit(X_train.reshape(-1, 1), Y_train)
Y_train_hat = model.predict(X_train.reshape(-1, 1))

In [ ]:
plt.scatter(X_train, Y_train, color="black")
plt.plot(X_train, Y_train_hat, color="red")
plt.title("Régression linéaire du nombre d'achats en magasin en fonction du revenu")

In [ ]:
print(model.intercept_, model.coef_, model.score(X_train.reshape(-1, 1), Y_train))

Equation de régression :

$$y_i = 0.46 + 2.21 * x_i$$

### Hypothèse 2 : exogénéité

In [ ]:
residuals = Y_train - Y_train_hat

plt.scatter(X_train, residuals)

In [ ]:
print(
    "Le coefficient de corrélation entre X et les residus vaut",
    pearsonr(X_train, residuals)[0],
    ". On a bien indépendance et donc exogénéité.",
)

### Hypothèse 3 : l'erreur suit une loi normale centrée i.e. E(ε) = 0

In [ ]:
import statistics
from scipy import stats

In [ ]:
plt.hist(residuals, density=True)

x = np.linspace(-7, 7, 100)
plt.plot(x, stats.norm.pdf(x, 0, 1))
plt.title("Histogramme des résidus en superposition avec la densité de la loi normale")
plt.show()

In [ ]:
print("La moyenne des résidus vaut", statistics.mean(residuals))
print("Mais les résidus ne suivent pas une loi normale")

In [ ]:
sm.qqplot(residuals, line="45")
print("On constate sur le qqplot que les points ne suivent pas la droite x = y")

In [ ]:
print(
    "Un test de shapiro, pour tester l'hypothèse de normalité, nous donne une pvalue de",
    stats.shapiro(residuals)[1],
    ". On rejette l'hypothèse nulle et on conclut que les résidus ne suivent pas une loi normale.",
)

### Hypothèse 4 : homoscédasticité

In [ ]:
def abline(slope, intercept):
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, "-")

In [ ]:
plt.plot(residuals, "bo")
plt.title("Nuage de points des résidus")
abline(0, 7)
abline(0, -7)

In [ ]:
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from statsmodels.compat import lzip

In [ ]:
fit = smf.ols("NumStorePurchases ~ Income", data=df_transforme).fit()

In [ ]:
bp_test = sms.het_breuschpagan(fit.resid, fit.model.exog)

In [ ]:
bp_names = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
print(lzip(bp_names, bp_test))
print("Il y a bien homoscédasticité.")

### Hypothèse 5 : absence d'autocorrélation

In [ ]:
from statsmodels.graphics.regressionplots import *

In [ ]:
sm.graphics.tsa.plot_acf(residuals)
print("On remarque une absence d'autocorrélation.")

### Distance de Cook

In [ ]:
influence = fit.get_influence()
cooks = influence.cooks_distance

In [ ]:
plt.scatter(X, cooks[0])
plt.xlabel("Revenus")
plt.ylabel("Distances de Cook")
plt.show()

In [ ]:
cooks_indexes = [i for i, x in enumerate(cooks[0] > 0.005) if x]
print(cooks_indexes)
# affiche les indexes des individus dont les distances de cook dépassent une certaine valeur

In [ ]:
influence_plot(fit)
print("")

### Score du modèle

#### Qualité d'ajustement

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
print(f"Le R² du modèle vaut {model.score(X_train.reshape(-1, 1), Y_train)}")

In [ ]:
print(f"MSE = {mean_squared_error(Y_train, Y_train_hat)}")
print(f"RMSE = {mean_squared_error(Y_train, Y_train_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_train, Y_train_hat)}")

#### Qualité de prédiction

Train / test split

In [ ]:
print(
    f"R² du modèle sur les données d'entraînement = {model.score(X_train.reshape(-1, 1), Y_train)}"
)
print(
    f"R² du modèle sur les données de test = {model.score(X_test.reshape(-1, 1), Y_test)}"
)

In [ ]:
Y_test_hat = model.predict(X_test.reshape(-1, 1))

In [ ]:
plt.scatter(X_test, Y_test, color="black")
plt.plot(X_test, Y_test_hat, color="red")
plt.title("Nombre d'achats en magasin en fonction du revenu : données test")

In [ ]:
print(f"MSE = {mean_squared_error(Y_test, Y_test_hat)}")
print(f"RMSE = {mean_squared_error(Y_test, Y_test_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_test, Y_test_hat)}")

In [ ]:
# todo: cross-validation

## Régression linéaire multiple

Pour les besoins de la régression linéaire, nous créons une variable Childhome étant égale à la somme de Teenhome et de Kidhome.  
Nous créons également des variables muettes (dummy variables) pour inclure les variables catégoriques `Education` et `Marital_status` à la régression.

In [ ]:
df_reg = df_transforme.copy()

In [ ]:
df_reg = pd.get_dummies(df_reg, columns=["Education", "Marital_Status"])

In [ ]:
df_reg.columns

Pour cette régression, on suppose que l'entreprise veut profiler au mieux les clients qui achètent dans le magasin. C'est pourquoi la variable d'intérêt pour la régression sera le `nombre d'achats en magasin`.  

In [ ]:
X = df_transforme[
    [
        "Income",
        "NumWebPurchases",
        "NumDealsPurchases",
        "NumWebVisitsMonth",
        "NumCatalogPurchases",
        "MntWines",
        "MntFruits",
        "MntMeatProducts",
        "MntFishProducts",
        "MntSweetProducts",
        "MntGoldProds",
        "Year_Birth",
        "NbChildren",
        "Recency",
        "NbAcceptedCampaigns",
    ]
]
Y = df_transforme["NumStorePurchases"]

### Avec statsmodels

In [ ]:
model_multiple_sm = sm.OLS(Y, X).fit()

In [ ]:
model_multiple_sm.summary()

Qualité d'ajustement du modèle avant suppression de variables

In [ ]:
print(f"MSE = {mean_squared_error(Y, Y_hat)}")
print(f"RMSE = {mean_squared_error(Y, Y_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y, Y_hat)}")

Réduction du nombre de variables explicatives par les critères AIC / BIC

In [ ]:
def aic_OLS(X):  # entrer X le dataframe des variables explicatives
    aic_list = []
    for col in X.columns:
        X_temp = X.drop(col, axis=1)
        model_temp = sm.OLS(Y, X_temp).fit()
        aic_list.append([col, model_temp.aic])

    return aic_list

In [ ]:
def stepwise_OLS(X):
    current_aic = sm.OLS(Y, X).fit().aic  # aic avec les variables actuelles
    aic_list = aic_OLS(X)  # liste des aic par supression des variables une a une

    my_bool = 0  # pour indiquer si aucune variable n'est a enlever
    for i in range(
        len(aic_list)
    ):  # si l'aic diminue pour une variable supprimee, on enleve cette variable du df X
        if aic_list[i][1] < current_aic:
            del X[aic_list[i][0]]  # a l'interieur des crochets c'est une string
            my_bool = 1

    if my_bool == 0:
        print("Le stepwise est terminé.")

In [ ]:
stepwise_OLS(X)

In [ ]:
stepwise_OLS(X)

In [ ]:
model_multiple_sm = sm.OLS(Y, X).fit()
Y_hat = model_multiple_sm.predict(X)

In [ ]:
plt.scatter(Y, Y_hat)
plt.title("Valeurs prédites contre vraies valeurs")
abline(1, 0)

In [ ]:
model_multiple_sm.summary()

Qualité d'ajustement après suppression de variables

In [ ]:
print(f"MSE = {mean_squared_error(Y, Y_hat)}")
print(f"RMSE = {mean_squared_error(Y, Y_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y, Y_hat)}")

### Avec sci-kit learn : entraînement et tests sur des vraies valeurs

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
model_multiple_sk = LinearRegression().fit(X_train, Y_train)
Y_test_hat = model_multiple_sk.predict(X_test)
Y_train_hat = model_multiple_sk.predict(X_train)

In [ ]:
plt.scatter(Y_test, Y_test_hat)
plt.title("Valeurs prédites contres vraies valeurs (valeurs test)")
abline(1, 0)

Qualité d'ajustement

In [ ]:
print(f"MSE = {mean_squared_error(Y_train, Y_train_hat)}")
print(f"RMSE = {mean_squared_error(Y_train, Y_train_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_train, Y_train_hat)}")

Qualité de prédiction

In [ ]:
print(f"MSE = {mean_squared_error(Y_test, Y_test_hat)}")
print(f"RMSE = {mean_squared_error(Y_test, Y_test_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_test, Y_test_hat)}")

Extraction des coefficients du modèle de régression linéaire :

In [ ]:
coefficients = pd.DataFrame(
    model_multiple_sk.coef_, X.columns, columns=["Coefficients"]
)
print(coefficients)
print("Intercept", model_multiple_sk.intercept_)

## Regression GLM

In [ ]:
import scipy

In [ ]:
X = df_transforme[
    [
        "Income",
        "NumWebPurchases",
        "NumDealsPurchases",
        "NumWebVisitsMonth",
        "NumCatalogPurchases",
        "MntWines",
        "MntFruits",
        "MntMeatProducts",
        "MntFishProducts",
        "MntSweetProducts",
        "MntGoldProds",
        "Year_Birth",
        "NbChildren",
        "Recency",
        "NbAcceptedCampaigns",
    ]
]
Y = df_transforme["NumStorePurchases"]

In [ ]:
scipy.stats.probplot(Y, dist="poisson", sparams=(5), plot=plt)
plt.show()

In [ ]:
poisson_model = sm.GLM(Y, X, family=sm.families.Poisson()).fit()

In [ ]:
poisson_model.summary()

Réduction du nombre de variables grâce au critère AIC

In [ ]:
def aic_GLM(X, Y):  # entrer X le dataframe des variables explicatives
    aic_list = []
    for col in X.columns:
        X_temp = X.drop(col, axis=1)
        model_temp = sm.GLM(Y, X_temp, family=sm.families.Poisson()).fit()
        aic_list.append([col, model_temp.aic])

    return aic_list

In [ ]:
def stepwise_GLM(X, Y):
    current_aic = (
        sm.GLM(Y, X, family=sm.families.Poisson()).fit().aic
    )  # aic avec les variables actuelles
    aic_list = aic_GLM(X, Y)  # liste des aic par supression des variables une a une

    my_bool = 0  # pour indiquer si aucune variable n'est a enlever
    for i in range(
        len(aic_list)
    ):  # si l'aic diminue pour une variable supprimee, on enleve cette variable du df X
        if aic_list[i][1] < current_aic:
            del X[aic_list[i][0]]  # a l'interieur des crochets c'est une string
            my_bool = 1

    if my_bool == 0:
        print("Le stepwise est terminé.")

In [ ]:
stepwise_GLM(X_train, Y_train)

In [ ]:
stepwise_GLM(X_train, Y_train)

In [ ]:
poisson_model = sm.GLM(Y_train, X_train, family=sm.families.Poisson()).fit()
Y_test_hat = poisson_model.predict(X_test)

In [ ]:
plt.scatter(Y, Y_hat)
plt.title("Valeurs prédites contres vraies valeurs (valeurs test)")
abline(1, 0)

In [ ]:
print(f"MSE = {mean_squared_error(Y, Y_hat)}")
print(f"RMSE = {mean_squared_error(Y, Y_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y, Y_hat)}")

## Régression polynomiale

In [ ]:
X = df_transforme[
    [
        "Income",
        "NumWebPurchases",
        "NumDealsPurchases",
        "NumWebVisitsMonth",
        "NumCatalogPurchases",
        "MntWines",
        "MntFruits",
        "MntMeatProducts",
        "MntFishProducts",
        "MntSweetProducts",
        "MntGoldProds",
        "Year_Birth",
        "Childhome",
        "Recency",
        "NbAcceptedCampaigns",
    ]
]
Y = df_transforme["NumStorePurchases"]

In [ ]:
# Transformer les données en matrice de caractéristiques polynomiales
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_poly, Y, test_size=0.5)

In [ ]:
# Entraîner un modèle de régression linéaire sur les données transformées
polynomial_model = LinearRegression().fit(X_train, Y_train)

In [ ]:
Y_test_hat = polynomial_model.predict(X_test)
Y_train_hat = polynomial_model.predict(X_train)

In [ ]:
plt.scatter(Y_test_hat, Y_test)
plt.title("Valeurs prédites contres vraies valeurs (valeurs test)")
abline(1, 0)

In [ ]:
print(f"MSE = {mean_squared_error(Y_test, Y_test_hat)}")
print(f"RMSE = {mean_squared_error(Y_test, Y_test_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_test, Y_test_hat)}")

In [ ]:
print(f"MSE = {mean_squared_error(Y_train, Y_train_hat)}")
print(f"RMSE = {mean_squared_error(Y_train, Y_train_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_train, Y_train_hat)}")

On constate que le modèle polynomial s'ajuste mieux aux données d'entraînement, mais est moins bon pour prédire les valeurs de test. Plus le degré des polynômes augmente, plus cet écart s'empire.

Jusqu'alors, le meilleur résultat a été obtenu par la régression linéaire multiple, avec élimination des variables par critères AIC.

# Autres modèles

## Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis()

In [ ]:
lda.fit(X_train, y_train)

In [ ]:
y_pred = lda.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=lda.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=lda.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lda.classes_)
disp.plot()

## Arbre de décision

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=model.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(random_state=0)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=model.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()

## XGBoost

In [ ]:
import xgboost

In [ ]:
model = xgboost.XGBClassifier(random_state=0)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=model.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()

## Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
model = VotingClassifier(
    estimators=[
        # ('lr', LogisticRegression(random_state=0)),
        ("dt", DecisionTreeClassifier(random_state=0)),
        # ('rf', RandomForestClassifier(random_state=0)),
        # ('lda', LinearDiscriminantAnalysis()),
        ("xgb", xgboost.XGBClassifier(random_state=0)),
    ],
    voting="soft",
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=model.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()